In [1]:
import timm
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [2]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePalooza Data/Benchmarking/Beetlepalooza_beetles_image_only.csv"
df = pd.read_csv(file_path, sep="\t")
df = df[['ImageFilePath', 'Genus']]
df.head(2) 

,ImageFilePath,Genus
0,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius
1,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = timm.create_model(
    model_name="hf-hub:1aurent/resnet50.lunit_swav",
    pretrained=True,
).to(device).eval()

data_config = timm.data.resolve_model_data_config(model)

transforms = timm.data.create_transform(**data_config, is_training=False)


In [4]:
def extract_features(image_path) :
    
    image = Image.open(image_path).convert("RGB")
    
    image_tensor = transforms(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        features = model(image_tensor)
    
    return features.cpu().numpy()

In [5]:
X = np.vstack([extract_features(img) for img in tqdm(df["ImageFilePath"], desc="Processing ..") ])

le = LabelEncoder()

y = le.fit_transform(df["Genus"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


Processing ..: 100%|██████████| 11399/11399 [08:31<00:00, 22.29it/s]


In [6]:
seed = 99

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "SVMPolynomial": SVC(kernel="poly", degree=4),
    "SVMRadialBasis": SVC(kernel="rbf", degree=4),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=seed),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(256, 128), activation='logistic', max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}

for name, model in models.items():
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T

/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


NaiveBayes                | Acc: 41.18% | Prec: 58.44% | Rec: 41.18% | F1: 44.47% | Bal Acc: 36.80% | MCC: 0.3784


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


LogisticRegression        | Acc: 88.20% | Prec: 87.82% | Rec: 88.20% | F1: 87.82% | Bal Acc: 73.00% | MCC: 0.8668


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


NearestNeighbor           | Acc: 75.31% | Prec: 76.45% | Rec: 75.31% | F1: 74.23% | Bal Acc: 47.08% | MCC: 0.7214


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


RandomForest              | Acc: 73.33% | Prec: 74.22% | Rec: 73.33% | F1: 70.07% | Bal Acc: 37.85% | MCC: 0.6968
MLP_Baseline              | Acc: 89.56% | Prec: 89.27% | Rec: 89.56% | F1: 89.20% | Bal Acc: 68.09% | MCC: 0.8822


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,Genus,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_SVMPolynomial,Pred_SVMRadialBasis,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
1766,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Synuchus,Cratacanthus,Synuchus,Synuchus,Pterostichus,Synuchus,Calathus,Calathus,Synuchus
2629,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Agonum,Cratacanthus,Calathus,Agonum,Calathus,Calathus,Amara,Calathus,Agonum


In [8]:
metrics_df

,Model,Accuracy,Precision,Recall,F1-Score,Balanced Acc,MCC
NaiveBayes,NaiveBayes,0.411842,0.584388,0.411842,0.444737,0.368031,0.378421
LogisticRegression,LogisticRegression,0.882018,0.878224,0.882018,0.878215,0.730018,0.866767
SVMLinear,SVMLinear,0.895175,0.894919,0.895175,0.892482,0.748844,0.881551
SVMPolynomial,SVMPolynomial,0.509649,0.710842,0.509649,0.491082,0.291156,0.478953
SVMRadialBasis,SVMRadialBasis,0.856579,0.854336,0.856579,0.844812,0.500254,0.837645
NearestNeighbor,NearestNeighbor,0.75307,0.764486,0.75307,0.742279,0.470809,0.721447
RandomForest,RandomForest,0.733333,0.742198,0.733333,0.70069,0.378473,0.696791
MLP_Baseline,MLP_Baseline,0.895614,0.892747,0.895614,0.892009,0.680894,0.882193


In [9]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/38.SwAV-linear-probing-genus.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/38.SwAV-linear-probing-genus-metrics.csv", index=False)